In [ ]:
from langchain_community.document_loaders import (
    PythonLoader,
    DirectoryLoader,
    TextLoader,
)
import os, json
from langchain_groq import ChatGroq
from langchain_community.embeddings import OllamaEmbeddings
from langchain_cohere.embeddings import CohereEmbeddings
from langchain_core.prompts import PromptTemplate
from langchain_community.agent_toolkits import FileManagementToolkit

groq_api_key = "your Groq api key"

llm = ChatGroq(
    groq_api_key=groq_api_key,
    model_name="llama3-groq-70b-8192-tool-use-preview",
    temperature=0.0,
)


embeddings_obj = CohereEmbeddings(
    model="embed-english-v3.0",
    cohere_api_key="cdu2whMVXxqmnCTG8wAIqd35ghLPgCnCKG7b8liN",
)


# loader=TextLoader('summary_all.txt')


# text_data=loader.load()


# text_data

In [ ]:
# from langchain.text_splitter import RecursiveCharacterTextSplitter

# text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=20)
# text_splitted = text_splitter.split_documents(text_data)

In [ ]:
# from langchain_community.vectorstores import FAISS

# db=FAISS.from_documents(text_splitted,embeddings_obj)

In [ ]:
from langchain_community.vectorstores import FAISS

db = FAISS.load_local(
    "Deep_learning_Java_Codes", embeddings_obj, allow_dangerous_deserialization=True
)
retriever = db.as_retriever()
retriever

In [ ]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "Vector_database",
    description=(
        """This tool retrieves documents file from a vector database based on the user's query. 
        Analyze the content of the user's query and match it with the most relevant documents in the database. 
        If the content and context of the query closely align with the documents' subject matter, 
        provide the corresponding file path where the matching document is stored by using the context passed in retriever object"""
    ),
)

In [ ]:
from langchain import hub


prompt_react = hub.pull("hwchase17/react-chat")
prompt_react

In [ ]:
from langchain_community.tools import ShellTool

shell_tool = ShellTool(
    name="Intermediate Answer",
    description="This tool is used for the commanding into windows os terminal and getting back the results of all types (eg.'grep','cls')",
    handle_tool_error=True,
    handle_validation_error=True,
    response_format="content",
)

In [ ]:
import os
from langchain.tools import Tool

def find_files_by_name(filename: str):
    root_dir = "C:\\Users\\Rudra\\Desktop"
    matches = []

    def search_directory(directory):
        with os.scandir(directory) as entries:
            for entry in entries:
                if entry.is_file() and entry.name == filename:
                    matches.append(entry.path)
                elif entry.is_dir():
                    search_directory(entry.path)

    search_directory(root_dir)
    if matches!=[]:
        return matches
    else:
        return "Sorry file couldn't found, please try again with some other clues...."

file_finder = Tool.from_function(
    func=find_files_by_name,
    name="file_finder",
    description="A tool for finding the file by their filename in 'C:\\Users\\Rudra\\Desktop' directory takes 2 input directory path and filename as input in it",
)

In [ ]:
tools = [retriever_tool, file_finder]
tools

In [ ]:
template = """Answer the following questions as best you can. You have access to the following tools:
{tools}
Use the following format:
Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question
Begin!
Question: {input}
Thought:{agent_scratchpad}"""
prompt_react = PromptTemplate.from_template(template)

In [ ]:
from langchain.agents import create_react_agent
from langchain.chains.conversation.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key="history", return_messages=True)
react_agent = create_react_agent(llm, tools, prompt_react)

In [ ]:
from langchain.agents import AgentExecutor, AgentType

agent_executor = AgentExecutor(
    agent=react_agent,
    tools=tools,
    verbose=True,
    return_intermediate_steps=True,
    handle_parsing_errors=True,
    memory=memory,
    max_execution_time=30
)
agent_executor

In [ ]:
import warnings

warnings.filterwarnings("ignore")
agent_executor.invoke({"input": "give the list of filepaths of files which contains the code for tower of hanoy problem in java"})["output"]